In [1]:
library(Seurat)
library(data.table)

Attaching SeuratObject



In [63]:
i=0
expdf = as.data.frame(t(fread(paste0('/home/test2/hECA_data/hECA_exprs_',i,'.csv'))))
expdf = expdf[-1,]
metadf = as.data.frame(fread(paste0('/home/test2/hECA_data/hECA_metadata_',i,'.csv')))

In [64]:
for (i in 1:18){
    message(paste0('/home/test2/hECA_data/hECA_exprs_',i,'.csv'))
    tmpexpdf = as.data.frame(t(fread(paste0('/home/test2/hECA_data/hECA_exprs_',i,'.csv'))))
    tmpexpdf = tmpexpdf[-1,]
    tmpmetadf = as.data.frame(fread(paste0('/home/test2/hECA_data/hECA_metadata_',i,'.csv')))
    expdf = cbind(expdf,tmpexpdf)
    metadf = rbind(metadf,tmpmetadf)    
}


/home/test2/hECA_data/hECA_exprs_1.csv

/home/test2/hECA_data/hECA_exprs_2.csv

/home/test2/hECA_data/hECA_exprs_3.csv

/home/test2/hECA_data/hECA_exprs_4.csv

/home/test2/hECA_data/hECA_exprs_5.csv

/home/test2/hECA_data/hECA_exprs_6.csv

/home/test2/hECA_data/hECA_exprs_7.csv

/home/test2/hECA_data/hECA_exprs_8.csv

/home/test2/hECA_data/hECA_exprs_9.csv

/home/test2/hECA_data/hECA_exprs_10.csv

/home/test2/hECA_data/hECA_exprs_11.csv

/home/test2/hECA_data/hECA_exprs_12.csv

/home/test2/hECA_data/hECA_exprs_13.csv

/home/test2/hECA_data/hECA_exprs_14.csv

/home/test2/hECA_data/hECA_exprs_15.csv

/home/test2/hECA_data/hECA_exprs_16.csv

/home/test2/hECA_data/hECA_exprs_17.csv

/home/test2/hECA_data/hECA_exprs_18.csv



In [85]:
row.names(metadf) <- metadf$cid
metadf = metadf[,1:18]
colnames(expdf) = rownames(metadf)
ref_obj <- CreateSeuratObject(expdf,  meta.data = metadf)

In [89]:
saveRDS(ref_obj, file = "Neuron_hECA_reference.rds")

In [47]:
data.matrix <- as.data.frame(fread("/home/test2/UniformedExpression.csv"))
row.names(data.matrix) <- data.matrix$V1
data.matrix = data.matrix[,-1]
cellid = colnames(data.matrix)
samp.devStage <- data.frame(samp.devStage = ifelse(grepl("^Fetal",cellid),"Fetal","Adult"))
rownames(samp.devStage) = cellid

# create Seurat object
query_obj <- CreateSeuratObject(counts = as.matrix(data.matrix),  meta.data = samp.devStage)

In [91]:
# QC
mito_genes <- rownames(query_obj)[grep("^[Mm][Tt]-",rownames(query_obj))]
query_obj <- PercentageFeatureSet(query_obj, "^MT-", col.name = "percent_mito")
ribo_genes <- rownames(query_obj)[grep("^RP[SL]",rownames(query_obj))]
query_obj <- PercentageFeatureSet(query_obj, "^RP[SL]", col.name = "percent_ribo")
hemoglobin_genes <- rownames(query_obj)[grep("^HB[^(P)]",rownames(query_obj))]
query_obj <- PercentageFeatureSet(query_obj, "^HB[^(P)]", col.name = "percent_hb")
selected_c <- WhichCells(query_obj, expression = nFeature_RNA > 200)# for UMI
selected_f <- rownames(query_obj)[Matrix::rowSums(query_obj) > 3]
query_obj.filt <- subset(query_obj, features = selected_f, cells = selected_c)

An object of class Seurat 
16614 features across 27412 samples within 1 assay 
Active assay: RNA (16614 features, 0 variable features)

In [95]:
saveRDS(query_obj.filt, file = "Neuron_query.rds")

In [ ]:
# 1min
refrds = readRDS('/home/hms/workspace/Neuron_hECA_reference.rds')
query_obj.filt = readRDS('/home/hms/workspace/Neuron_query.rds')

In [ ]:
# 7min
gene.use = intersect(row.names(refrds),row.names(query_obj.filt))
refrds.filt <- subset(refrds, features = gene.use)
ct.ref <- refrds.filt$cell_type
trainedR <- trainSingleR(refrds.filt@assays$RNA@counts, ct.ref, de.method = "wilcox")

In [ ]:
# 3h
predict <- classifySingleR(query_obj.filt@assays$RNA@data,trainedR)
saveRDS(predict,'SingleR_transfer.rds')

In [96]:
predict = readRDS('/home/test2/SingleR_transfer.rds')

In [100]:
predict

DataFrame with 27412 rows and 5 columns
                                            scores          first.labels
                                          <matrix>           <character>
Ex3e            0.1393036:0.0797517:-0.0189485:...     Excitatory neuron
Ex2             0.0903021:0.0257629:-0.1092682:...  PV inhibitory neuron
In1b            0.1796569:0.1372260: 0.0113191:... VIP inhibitory neuron
Oligo           0.1466988:0.1289090: 0.0914283:... VIP inhibitory neuron
Ex1             0.0731952:0.0235708:-0.0988549:...     Excitatory neuron
...                                            ...                   ...
Microglia.312 0.0198871:-0.01109635:-0.1011555:...     Excitatory neuron
Microglia.313 0.0698576: 0.02837352:-0.0333879:...     Excitatory neuron
Microglia.314 0.0832008: 0.08401538: 0.0256835:...     Excitatory neuron
Microglia.315 0.0407454: 0.00299379:-0.0794683:...     Excitatory neuron
Microglia.316 0.1002097: 0.06342319:-0.0242063:...  PV inhibitory neuron
           